# Firework Simulation! Now With Particles!
A particle based fireworks simulation inspired by Daniel Shiffman's [tutorial](https://www.youtube.com/watch?v=CKeyIbT3vXI "The Coding Train (YouTube)"), using Python 3 and the [Pyglet](https://pyglet.readthedocs.io/en/pyglet-1.3-maintenance/index.html# "Pyglet Reference") library (for drawing).

Get Pyglet by running `pip install pyglet`

In [1]:
# Imports
import pyglet, random, math

TIME_STEP = 1/60 #physics calculation interval in seconds

WINDOW_SIZE = (800, 600)
BACKGROUND_COLOR = [0, 0, 15]*2 + [10, 0, 30]*2

PARTICLE_RATE   = 2  # Particles spawned per second
PARTICLE_RADIUS = 3  # Default radius of particles
PARTICLE_SIDES  = 6  # Number of sides on particle polygon

FIREWORK_ACC = (0, -450)              # Initial x,y acceleration of fireworks inc. GRAVITY
FIREWORK_VEL_LIM = (-75,75,500,700) # Firework velocity limits (xmin, xmax, ymin, ymax)
FIREWORK_BOOM_VEL = -50               # Upper y-velocity threshold at which to explode
FIREWORK_FADE_DURATION = 45           # Number of updates over which to fade
LAUNCH_WIDTH = 0.75                   # Central fraction of window width to launch from

EXPL_PART_SIZE = [3, 6, 10]           # Radii of explosion particles (all used in equal proportion)
EXPL_PARTICLES = 100                  # Number of particles to spawn on explosion
EXPL_PART_ACC  = (0, -150)            # Explosion particle x,y acceleration (lower gravity for aesthetics)
EXPL_VELOCITY  = 100                  # Magnitude of resultant particle velocity
EXPL_VARIATION = [0.1, 0.66, 1.5]     # Percentage variation from velocity
EXPL_VARIATION_WEIGHTS = [2, 8, 10]   # Cumulative weights for variation to occur

# Pyglet window initialation
window = pyglet.window.Window(WINDOW_SIZE[0], WINDOW_SIZE[1], caption='Fireworks')
# OpenGL configuration to handle particle opacity
pyglet.gl.glEnable(pyglet.gl.GL_BLEND)
pyglet.gl.glBlendFunc(pyglet.gl.GL_SRC_ALPHA, pyglet.gl.GL_ONE_MINUS_SRC_ALPHA)

# Initialisation of global particles list
particles = []

### Colour Handling
A few functions to make colour handling slightly easier.  All Colours are use the RGBA (0-255) format, and are stored in a 4 item list.

In [2]:
#returns a random RGBA colour with 0 alpha (transparent)
def randColor():
    return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), 0]

# accepts an RGB/RGBA colour and increases all color channels by a factor of 3, and sets opacity to 100%
def brightenColor(col):
    return [int(min(col[0]*3,255)), int(min(col[1]*3,255)), int(min(col[2]*3,255)), 255]

### Particle Class
Particles have individual x,y positions, velocities and accelerations, as well as an RGBA colour and a size in pixels. Additionally, particles may be set to explode (disappear instantly) or with otherwise fade out over time.

Position, velocity and acceleration are all input as an x,y tuple, but are internally stored as distinct values.

Particles are drawn as a fan of triangles around a central vertex. The outer vertices have the base color, while the central vertex is the base colour brightened.

In [3]:
class Particle:
    def __init__(self, xyPos, xyVel, xyAcc, color, size=PARTICLE_RADIUS, explode=False):
        # Initialise position, velocity and acceleration
        (self.xPos, self.yPos) = xyPos
        (self.xVel, self.yVel) = xyVel
        (self.xAcc, self.yAcc) = xyAcc
        # Save base color (for spawning children)
        self.baseCol = color
        # Build list of vertices' color values (4 elements per vertex R, G ,B, A, R, G, ...)
        self.col = brightenColor(color) + color * (PARTICLE_SIDES + 1)
        # Initialise coordinates of mesh vertices. First items in list are particle x-position and y-position
        self.verts = list(xyPos)
        for ii in range(PARTICLE_SIDES + 1):
            angle = ii * 2 * math.pi / PARTICLE_SIDES
            self.verts.append(self.xPos + size * math.cos(angle))
            self.verts.append(self.yPos + size * math.sin(angle))
        # Initialise other properties
        self.explode = explode
        self.lifespan = 0
    
    # Updates attributes of particle as if 'time' seconds have passed
    def update(self, time):
        # Update particle position
        self.xPos += self.xVel*time
        self.yPos += self.yVel*time
        
        # Update vertices' positions
        self.verts[0], self.verts[1] = self.xPos, self.yPos
        for pos in range(2, 2*(PARTICLE_SIDES+2), 2):
            self.verts[pos]   += self.xVel*time
            self.verts[pos+1] += self.yVel*time

        # Update acceleration 
        self.xVel = (self.xVel + self.xAcc*time)
        self.yVel = (self.yVel + self.yAcc*time)
        # Increment age
        self.lifespan += 1
        # Fade particles by reducing all alpha values (loops through every 4th item in colour array)
        if(not self.explode):
            alpha = max(0, round(255 * (1 - self.lifespan/FIREWORK_FADE_DURATION)))
            for ii in range(3, len(self.col), 4*(PARTICLE_SIDES + 2)): self.col[ii] = alpha

    # Draws the particle using a triangle fan.
    # Currently a source of inefficiency -> all particles call draw method
    def draw(self):
        pyglet.graphics.draw(PARTICLE_SIDES+2, pyglet.gl.GL_TRIANGLE_FAN,  ('v2f', self.verts), ('c4B', self.col))

### Spawning New Particles
Some functions for creating new fireworks and explosion particles.

In [4]:
# Generates/"Launches" a new firework.
# 'elapsedTime' is automaticaly set as the duration since the function was last called
def newFirework(elapsedTime):
    # Don't launch firework if performance is much (20%) lower than target performance
    if elapsedTime < 1.2 / PARTICLE_RATE:
        global particles
        pos = (random.uniform(WINDOW_SIZE[0]*(0.5-LAUNCH_WIDTH/2), WINDOW_SIZE[0]*(0.5 + LAUNCH_WIDTH/2)), 0)
        vel = (random.uniform(FIREWORK_VEL_LIM[0], FIREWORK_VEL_LIM[1]), # x-velocity
               random.uniform(FIREWORK_VEL_LIM[2], FIREWORK_VEL_LIM[3])) # y-velocity
        particles.append(Particle(pos, vel, FIREWORK_ACC, randColor(), explode=True))

# Spawns a new explosion particle with position and color inherited from
def genExplParticle(parent, size, rVariation):
    angle, mag = random.uniform(0, 2*math.pi), EXPL_VELOCITY*random.uniform(1-rVariation, 1+rVariation)
    #inherits some velocity from parent (currently only x-velocity for aesthetic reasons)
    vel = (mag*math.cos(angle) + parent.xVel, mag*math.sin(angle))
    return Particle((parent.xPos, parent.yPos), vel, EXPL_PART_ACC, parent.baseCol, size=size, explode=False)

### Main Loop
This function is repeatedly called, and is the driver of all particle updates. Function all checks whether or not a firework has exploded/faded, disposes of "dead" particles and spawns the explosion particles.

In [5]:
def updateParticles(elapsedTime):
    # Give access to global variable
    global particles
    # Reset window and draw background plane
    window.clear()
    pyglet.graphics.draw(4, pyglet.gl.GL_QUADS,  ('v2i', (0, WINDOW_SIZE[1], WINDOW_SIZE[0], WINDOW_SIZE[1], WINDOW_SIZE[0], 0, 0, 0)),('c3B', BACKGROUND_COLOR))
    
    # Iterate over all particles and update
    for ii in range(len(particles)):
        particles[ii].update(elapsedTime)
        particles[ii].draw()
        # Handle exploding/fading particles
        if(particles[ii].explode and particles[ii].yVel < FIREWORK_BOOM_VEL):
            # Generate explosion particles
            shape = random.choices(EXPL_VARIATION, cum_weights=EXPL_VARIATION_WEIGHTS, k=1)[0]
            for num in range(EXPL_PARTICLES): particles.append(genExplParticle(particles[ii], random.choice(EXPL_PART_SIZE),shape))
            
            # Mark parent particle as dead
            particles[ii] = None
            
        elif((not particles[ii].explode) and particles[ii].lifespan > FIREWORK_FADE_DURATION):
            # Mark particle as dead
            particles[ii] = None
            
    #removes "null" particles from list in a single batch
    particles = list(filter(None, particles))

### Launch the Fireworks!

In [6]:
#scheduling physics calculations and firework 'launches'
pyglet.clock.schedule_interval(updateParticles, TIME_STEP)
pyglet.clock.schedule_interval(newFirework, 1/PARTICLE_RATE)

#run simulation
pyglet.app.run()

### Improvements
##### Performance
Currently performance is somewhat tollerable at best. This is largely due to the high cost of generating all the random values required to initialise explosion particles and the general volume of vertex redraws required as particle numbers climb. These issues could be addressed thusly:
1. Maintain a global vertices list and draw all particles in a single batch (rather than one by one)
1. Creating and drawing pools of predetermined explosion particles/radii/colours rather than generating all random numbers on demand

##### Visual Quality
There are a multitude of aesthetic features that could be added to the simulation. This includes particle effects (twinkles, colour transitions etc), different explosion shapes, multi-stage charges and most obviously, the third dimension! These would require *significantly* more effort, however only small modifications to the base class would be required, alongside the performance changes listed above.